In [1]:
# Yahoo Finance Data Acquisition of Required Data
#Target Period: 2006-01-01 to 2025-06-30

#Imports
#Import Yahoo Finance data library for data acquisition
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
#Define instruments and Date range
start = "2006-01-01"
end = "2025-06-30"

# S&P500 ETF (proxy)
df_spx_raw = yf.download("SPY", start=start, end=end)

# Dollar Index (Yahoo ticker: DX-Y.NYB)
df_dxy = yf.download("DX-Y.NYB", start=start, end=end)

# WTI Oil Futures (Yahoo ticker: CL=F)
df_wti = yf.download("CL=F", start=start, end=end)

print(df_spx_raw.head())
print(df_dxy.head())
print(df_wti.head())

C:\Users\josep\AppData\Local\Temp\ipykernel_76596\3065575225.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_spx_raw = yf.download("SPY", start=start, end=end)
[*********************100%***********************]  1 of 1 completed
C:\Users\josep\AppData\Local\Temp\ipykernel_76596\3065575225.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_dxy = yf.download("DX-Y.NYB", start=start, end=end)
[*********************100%***********************]  1 of 1 completed
C:\Users\josep\AppData\Local\Temp\ipykernel_76596\3065575225.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_wti = yf.download("CL=F", start=start, end=end)
[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open    Volume
Ticker            SPY        SPY        SPY        SPY       SPY
Date                                                            
2006-01-03  87.721687  87.929397  86.122343  86.676231  73256700
2006-01-04  88.137154  88.268698  87.721735  87.832514  51899600
2006-01-05  88.192543  88.337938  87.846364  88.033304  47307500
2006-01-06  88.926407  89.023336  88.178659  88.635618  62885900
2006-01-09  89.154869  89.355648  88.884850  88.912540  43527400
Price           Close       High        Low       Open   Volume
Ticker       DX-Y.NYB   DX-Y.NYB   DX-Y.NYB   DX-Y.NYB DX-Y.NYB
Date                                                           
2006-01-03  89.839996  90.940002  89.779999  90.750000        0
2006-01-04  89.139999  89.860001  89.010002  89.730003        0
2006-01-05  89.330002  89.629997  89.180000  89.269997        0
2006-01-06  88.849998  89.629997  88.800003  89.349998        0
2006-01-09  89.250000  89.449997

# Data Preprocessing & Transformation

In [5]:
#Import required libraries
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import adfuller

#Load and confirm the raw datasets
print("SPY shape:", df_spx_raw.shape)
print("DXY shape:", df_dxy.shape)
print("WTI shape:", df_wti.shape)

#Standardise column names and isolate 'Close' column
df_spx = df_spx_raw[['Close']].rename(columns={'Close': 'SPY_Close'})
df_dxy = df_dxy[['Close']].rename(columns={'Close': 'DXY_Close'})
df_wti = df_wti[['Close']].rename(columns={'Close': 'WTI_Close'})

#Merge and clean datasets
df = df_spx.join([df_dxy, df_wti], how='inner')
df = df.dropna()
print("Merged dataframe shape:", df.shape)

#Compute returns and changes
df['SPY_log_ret'] = np.log(df['SPY_Close'] / df['SPY_Close'].shift(1)) * 100
df['SPY_price_change'] = df['SPY_Close'].diff()
df['SPY_p_change'] = df['SPY_Close'].pct_change() * 100

df['DXY_ret'] = np.log(df['DXY_Close'] / df['DXY_Close'].shift(1)) * 100
df['WTI_ret'] = np.log(df['WTI_Close'] / df['WTI_Close'].shift(1)) * 100

df = df.dropna()

SPY shape: (4902, 5)
DXY shape: (4905, 5)
WTI shape: (4901, 5)
Merged dataframe shape: (4898, 3)


C:\Users\josep\anaconda3\envs\LSTM_Research_Project\Lib\site-packages\pandas\core\internals\blocks.py:395: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


# Time Series Splitting & Validation

# Static Chronological Split

In [9]:
# ================================================================
# 3.2A STATIC CHRONOLOGICAL SPLIT
# ================================================================

# Function for chronological splitting of time series data
def chronological_split(df, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    """
    Chronologically splits the dataset into train, validation, and test sets.
    Ensures no look-ahead bias by using the time order of observations.

    Parameters:
    - df: pandas DataFrame with DateTime index
    - train_ratio: float, proportion of training data
    - val_ratio: float, proportion of validation data
    - test_ratio: float, proportion of testing data

    Returns:
    - train_df, val_df, test_df
    """
    # Validate ratios
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-9, "Ratios must sum to 1."

    n = len(df)
    i_train_end = int(n * train_ratio)
    i_val_end = int(n * (train_ratio + val_ratio))

    train_df = df.iloc[:i_train_end].copy()
    val_df = df.iloc[i_train_end:i_val_end].copy()
    test_df = df.iloc[i_val_end:].copy()

    print(f"Total samples: {n}")
    print(f"Train: {len(train_df)} ({len(train_df)/n:.1%}), "
          f"Validation: {len(val_df)} ({len(val_df)/n:.1%}), "
          f"Test: {len(test_df)} ({len(test_df)/n:.1%})")

    return train_df, val_df, test_df


# ---------------------------------------------------------------
# Apply chronological split to your processed dataframe
# ---------------------------------------------------------------
train_df, val_df, test_df = chronological_split(df, 0.8, 0.1, 0.1)

# Confirm the split visually
print("\nTrain range:", train_df.index.min(), "→", train_df.index.max())
print("Validation range:", val_df.index.min(), "→", val_df.index.max())
print("Test range:", test_df.index.min(), "→", test_df.index.max())

# Display sample data from each set
print("\nSample from training data:")
print(train_df.tail())

print("\nSample from validation data:")
print(val_df.head())

print("\nSample from test data:")
print(test_df.head())

Total samples: 4895
Train: 3916 (80.0%), Validation: 489 (10.0%), Test: 490 (10.0%)

Train range: 2006-01-04 00:00:00 → 2021-08-03 00:00:00
Validation range: 2021-08-04 00:00:00 → 2023-07-14 00:00:00
Test range: 2023-07-17 00:00:00 → 2025-06-27 00:00:00

Sample from training data:
Price        SPY_Close  DXY_Close  WTI_Close SPY_log_ret SPY_price_change  \
Ticker             SPY   DX-Y.NYB       CL=F                                
Date                                                                        
2021-07-28  413.542023  92.320000  72.389999   -0.041015        -0.169647   
2021-07-29  415.257111  91.860001  73.620003    0.413874         1.715088   
2021-07-30  413.240509  92.169998  73.949997   -0.486810        -2.016602   
2021-08-02  412.373535  92.050003  71.260002   -0.210019        -0.866974   
2021-08-03  415.728302  92.080002  70.559998    0.810235         3.354767   

Price      SPY_p_change   DXY_ret   WTI_ret  
Ticker                                       
Date     

Create functions: AWT and Indicators, scaling and rolling windows

# Rolling Window (Walk-Forward Validation) and Adaptive Wavelength Transform (AWT)

# Flatten multi-index columns and apply wavelet denoising helper

In [15]:
# ================================================================
# ✅ FIXED: Walk-Forward Validation with AWT Features + Scaling
# ================================================================
import pywt
import ta
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# ================================================================
# 1️⃣  Utility: Flatten MultiIndex columns
# ================================================================
def flatten_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure single-level, clean column names."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            '_'.join([str(c) for c in tup if c is not None]).strip()
            for tup in df.columns
        ]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    # Remove duplicates if flattening causes any
    df = df.loc[:, ~pd.Index(df.columns).duplicated()]
    return df


# ================================================================
# 2️⃣  Wavelet Denoising Helper
# ================================================================
def wavelet_denoise_series(x: pd.Series,
                           wavelet: str = "db2",
                           level: int = 2,
                           mode: str = "soft") -> pd.DataFrame:
    """
    Simple wavelet decompose + denoise returning smooth & detail as DataFrame.
    """
    x = pd.Series(x).astype(float)
    coeffs = pywt.wavedec(x.values, wavelet=wavelet, level=level)
    # Universal threshold based on noise estimation
    sigma = np.median(np.abs(coeffs[-1])) / 0.6745 if len(coeffs[-1]) else 0.0
    thr = sigma * np.sqrt(2 * np.log(len(x)))
    coeffs_thr = [coeffs[0]] + [pywt.threshold(c, thr, mode=mode) for c in coeffs[1:]]
    x_s = pywt.waverec(coeffs_thr, wavelet)[:len(x)]  # smoothed component
    x_d = x.values[:len(x_s)] - x_s  # detail component
    return pd.DataFrame({"SPY_s": x_s, "SPY_d": x_d}, index=x.index)


# Build AWT features and 21 financial indicators

In [18]:
# ================================================================
# 3️⃣  Compute AWT + Features + Target
# ================================================================
# ------------------------------------------------
# Robust AWT + features + target (auto-make SPY_log_ret if missing)
# ------------------------------------------------
def _first_present(colnames, df_cols):
    for c in colnames:
        if c in df_cols:
            return c
    return None

def compute_awt_features(df_slice: pd.DataFrame) -> pd.DataFrame:
    """
    Build AWT-based features + 21 indicators safely.
    Requires SPY OHLCV if available; will fall back where possible.
    Produces:
      SPY_s, SPY_d, and 21 indicators incl. MACD, RSI, ADX, OBV, MFI, WVAD, etc.
    """
    # -------- helpers --------
    def _flatten(df):
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = ['_'.join([str(c) for c in tup if c is not None]).strip()
                          for tup in df.columns]
        else:
            df.columns = [str(c).strip() for c in df.columns]
        return df.loc[:, ~pd.Index(df.columns).duplicated()]

    def _find_col(df, *must_contain):
        must = [m.lower() for m in must_contain]
        for c in df.columns:
            lc = str(c).lower()
            if all(m in lc for m in must):
                return c
        # simple fallbacks for common single-level names
        lut = {
            ('spy', 'open'):   ['SPY_Open', 'Open', 'Open_SPY'],
            ('spy', 'high'):   ['SPY_High', 'High', 'High_SPY'],
            ('spy', 'low'):    ['SPY_Low', 'Low', 'Low_SPY'],
            ('spy', 'close'):  ['SPY_Close', 'Close', 'Close_SPY'],
            ('spy', 'volume'): ['SPY_Volume', 'Volume', 'Volume_SPY'],
        }
        for k, alts in lut.items():
            if set(k) == set(must):
                for a in alts:
                    if a in df.columns:
                        return a
        return None

    def _wavelet_denoise(x, wavelet="db2", level=2, mode="soft"):
        x = pd.Series(x).astype(float)
        coeffs = pywt.wavedec(x.values, wavelet=wavelet, level=level)
        sigma = np.median(np.abs(coeffs[-1])) / 0.6745 if len(coeffs[-1]) else 0.0
        thr   = sigma * np.sqrt(2 * np.log(len(x)))
        coeffs_thr = [coeffs[0]] + [pywt.threshold(c, thr, mode=mode) for c in coeffs[1:]]
        x_s = pywt.waverec(coeffs_thr, wavelet)[:len(x)]
        x_d = x.values[:len(x_s)] - x_s
        return pd.DataFrame({"SPY_s": x_s, "SPY_d": x_d}, index=x.index)

    # -------- start --------
    df_f = _flatten(df_slice.copy())

    # Locate OHLCV columns (best effort)
    c_open   = _find_col(df_f, 'spy', 'open')
    c_high   = _find_col(df_f, 'spy', 'high')
    c_low    = _find_col(df_f, 'spy', 'low')
    c_close  = _find_col(df_f, 'spy', 'close')
    c_volume = _find_col(df_f, 'spy', 'volume')

    # Ensure log returns exist (needed for AWT)
    if 'SPY_log_ret' not in df_f.columns:
        if c_close is None:
            raise KeyError("compute_awt_features() needs SPY close prices to derive SPY_log_ret.")
        df_f['SPY_log_ret'] = np.log(df_f[c_close] / df_f[c_close].shift(1)) * 100

    # AWT on SPY_log_ret -> SPY_s (smooth), SPY_d (detail)
    s_hat = _wavelet_denoise(df_f['SPY_log_ret'])
    df_f = df_f.join(s_hat, how='left')

    # Choose price series for TA (prefer denoised)
    price = df_f['SPY_s'].fillna(df_f['SPY_log_ret'])

    # ---------------- 21 INDICATORS ----------------
    # 1–3 Trend MAs
    df_f['EMA20'] = ta.trend.EMAIndicator(close=price, window=20).ema_indicator()
    df_f['MA5']   = ta.trend.SMAIndicator(close=price, window=5).sma_indicator()
    df_f['MA10']  = ta.trend.SMAIndicator(close=price, window=10).sma_indicator()

    # 4 MACD
    df_f['MACD']  = ta.trend.MACD(close=price).macd()

    # 5 ROC(10)
    df_f['ROC']   = ta.momentum.ROCIndicator(close=price, window=10).roc()

    # 6–7 Momentum (6m, 12m)
    df_f['MTM6']  = price / price.shift(126) - 1
    df_f['MTM12'] = price / price.shift(252) - 1

    # 8 SMI (stochastic momentum index)
    if c_high and c_low:
        stoch = ta.momentum.StochasticOscillator(high=df_f[c_high], low=df_f[c_low],
                                                 close=price, window=14, smooth_window=3)
        df_f['SMI'] = stoch.stoch_signal()
    else:
        df_f['SMI'] = np.nan

    # 9 ATR(14)
    df_f['ATR'] = (ta.volatility.AverageTrueRange(
        high=df_f[c_high] if c_high else price,
        low=df_f[c_low] if c_low else price,
        close=price, window=14
    ).average_true_range())

    # 10–12 Bollinger
    boll = ta.volatility.BollingerBands(close=price, window=20, window_dev=2)
    df_f['BOLL_upper']     = boll.bollinger_hband()
    df_f['BOLL_lower']     = boll.bollinger_lband()
    df_f['BOLL_bandwidth'] = df_f['BOLL_upper'] - df_f['BOLL_lower']

    # 13 CCI(20)
    if c_high and c_low and c_close:
        df_f['CCI'] = ta.trend.CCIIndicator(
            high=df_f[c_high], low=df_f[c_low], close=df_f[c_close], window=20
        ).cci()
    else:
        df_f['CCI'] = np.nan

    # 14 WVAD (needs OHLCV)
    if c_open and c_high and c_low and c_close and c_volume:
        rng = (df_f[c_high] - df_f[c_low]).replace(0, np.nan)
        df_f['WVAD'] = ((df_f[c_close] - df_f[c_open]) / (rng + 1e-10)) * df_f[c_volume]
    else:
        df_f['WVAD'] = np.nan

    # 15 RSI(14)
    df_f['RSI14'] = ta.momentum.RSIIndicator(close=price, window=14).rsi()

    # 16–17 Stochastic %K/%D (classic)
    if c_high and c_low:
        st = ta.momentum.StochasticOscillator(high=df_f[c_high], low=df_f[c_low],
                                              close=price, window=14, smooth_window=3)
        df_f['STO_K'] = st.stoch()
        df_f['STO_D'] = st.stoch_signal()
    else:
        df_f[['STO_K', 'STO_D']] = np.nan

    # 18 ADX(14)
    if c_high and c_low and c_close:
        df_f['ADX14'] = ta.trend.ADXIndicator(
            high=df_f[c_high], low=df_f[c_low], close=df_f[c_close], window=14
        ).adx()
    else:
        df_f['ADX14'] = np.nan

    # 19 OBV
    if c_close and c_volume:
        df_f['OBV'] = ta.volume.OnBalanceVolumeIndicator(
            close=df_f[c_close], volume=df_f[c_volume]
        ).on_balance_volume()
    else:
        df_f['OBV'] = np.nan

    # 20 MFI(14)
    if c_high and c_low and c_close and c_volume:
        df_f['MFI14'] = ta.volume.MFIIndicator(
            high=df_f[c_high], low=df_f[c_low],
            close=df_f[c_close], volume=df_f[c_volume], window=14
        ).money_flow_index()
    else:
        df_f['MFI14'] = np.nan

    # 21 Williams %R(14)
    if c_high and c_low and c_close:
        df_f['WILLR14'] = ta.momentum.WilliamsRIndicator(
            high=df_f[c_high], low=df_f[c_low], close=df_f[c_close], lbp=14
        ).williams_r()
    else:
        df_f['WILLR14'] = np.nan

    # Target (example: next-day log return)
    df_f['y_1d'] = df_f['SPY_log_ret'].shift(-1)

    # Final clean (drop rows introduced by windows/shifts)
    return df_f.dropna()


# Add Scaling Helper and Rolling Window Generator

In [21]:
# ================================================================
# 4️⃣  Scaling Helper
# ================================================================
def fit_transform_scale(X_train, X_val=None, X_test=None):
    """
    Fit StandardScaler on training set; transform others with same parameters.
    """
    scaler = StandardScaler()
    Xtr = scaler.fit_transform(X_train)
    Xva = scaler.transform(X_val) if X_val is not None else None
    Xte = scaler.transform(X_test) if X_test is not None else None
    return Xtr, Xva, Xte, scaler


# ================================================================
# 5️⃣  Rolling Window Generator
# ================================================================
def walk_forward_windows(df, train_span, eval_span, step):
    """
    Generator producing chronological (train_df, eval_df) pairs.
    """
    n = len(df)
    start = 0
    while True:
        tr_start = start
        tr_end   = tr_start + train_span
        ev_end   = tr_end + eval_span
        if ev_end > n:
            break
        yield df.iloc[tr_start:tr_end].copy(), df.iloc[tr_end:ev_end].copy()
        start += step


# Implement walk forward validation using pre-defined functions

In [24]:
# ================================================================
# 6️⃣  Walk-Forward Execution Loop
# ================================================================
# --- REPLACE your walk-forward loop with this safer version ---
TRAIN_SPAN = 1500      # ~6 years
EVAL_SPAN  = 60        # ~3 months
STEP       = 20        # ~1 month shift

feature_cols = None
target_col   = 'y_1d'

MIN_TR_ROWS  = 100   # guard: minimum rows required after dropna
MIN_EV_ROWS  = 10

all_preds, all_truth = [], []

made_plots = False  # Track if we’ve already plotted

for w, (df_tr_raw, df_ev_raw) in enumerate(walk_forward_windows(df, TRAIN_SPAN, EVAL_SPAN, STEP), start=1):
    df_tr_raw = flatten_cols(df_tr_raw)
    df_ev_raw = flatten_cols(df_ev_raw)

    # Build features on combined slice
    df_comb       = pd.concat([df_tr_raw, df_ev_raw], axis=0)
    df_feat_comb  = compute_awt_features(df_comb)

    # Split back to train/eval by index, THEN drop NaNs
    tr_end_idx = df_tr_raw.index[-1]
    df_feat_tr = df_feat_comb.loc[:tr_end_idx].dropna()
    df_feat_ev = df_feat_comb.loc[df_ev_raw.index[0]:df_ev_raw.index[-1]].dropna()

    # Window too small? (happens in first windows due to long lookbacks)
    if len(df_feat_tr) < MIN_TR_ROWS or len(df_feat_ev) < MIN_EV_ROWS:
        # optional: print(f"Skipping window {w}: train={len(df_feat_tr)}, eval={len(df_feat_ev)}")
        continue

    # Choose features once (exclude raw returns & targets)
    if feature_cols is None:
        exclude = {'SPY_log_ret', 'DXY_ret', 'WTI_ret'} | {c for c in df_feat_tr.columns if c.startswith('y_')}
        feature_cols = [c for c in df_feat_tr.columns if c not in exclude]

    X_tr = df_feat_tr[feature_cols].values
    y_tr = df_feat_tr[target_col].values
    X_ev = df_feat_ev[feature_cols].values
    y_ev = df_feat_ev[target_col].values

    # Another guard (just in case)
    if X_tr.shape[0] == 0 or X_ev.shape[0] == 0:
        # optional: print(f"Skipping window {w}: empty matrix after selection")
        continue

    # Scale using train stats only
    X_tr_s, _, X_ev_s, scaler = fit_transform_scale(X_tr, None, X_ev)

    # Fit & predict here when ready:
    # model.fit(X_tr_s, y_tr)
    # y_hat = model.predict(X_ev_s)
    # all_preds.append(y_hat); all_truth.append(y_ev)

print("✅ Walk-forward loop ran with NaN-safe slicing and window guards.")


✅ Walk-forward loop ran with NaN-safe slicing and window guards.


# Stationarity Tests (ADF & PP)

Stationarity: If ADF/PP p-values < 0.05, treat SPY_log_ret as stationary (set ARIMAX d=0).

Long memory:

R/S (Hurst H): H≈0.5 → short memory; H>0.5 → persistent/long memory; H<0.5 → anti-persistence.

GPH d: 0 ≤ d < 0.5 indicates long memory but stationary, motivating ARFIMA/FI-volatility (e.g., FIGARCH/FIEGARCH).

Next step connection: Use these diagnostics to justify (i) ARIMAX order search with d=0 for returns, and (ii) FIGARCH/EGARCH (or FIEGARCH via R) for volatility with a fractional component if d suggests long memory

In [27]:
# ===============================================================
# Stationarity Tests on SPY_log_ret
# - Augmented Dickey–Fuller (ADF)
# - Phillips–Perron (PP)
# ===============================================================
from arch.unitroot import PhillipsPerron

# ---------- Helper Functions ----------
def run_adf(x: pd.Series, autolag='AIC'):
    x = pd.Series(x).dropna()
    res = adfuller(x, autolag=autolag)
    return {
        'stat': res[0],
        'pvalue': res[1],
        'lags_used': res[2],
        'nobs': res[3],
        'crit': {'1%': res[4]['1%'], '5%': res[4]['5%'], '10%': res[4]['10%']},
    }

def run_pp(x: pd.Series, trend='c'):
    """
    Phillips–Perron unit root test.
    Compatible with arch versions that do not expose .bandwidth.
    """
    x = pd.Series(x).dropna()
    res = PhillipsPerron(x, trend=trend)
    return {
        'stat': res.stat,
        'pvalue': res.pvalue,
        'lags_used': getattr(res, 'lags', None),
        'crit': getattr(res, 'critical_values', {}),
        'summary': res.summary().as_text() if hasattr(res, 'summary') else None
    }

def stationarity_verdict(p):
    return "Reject unit root (stationary)" if p < 0.05 else "Fail to reject unit root (non-stationary)"

# ---------- Run Tests ----------
y_ret = df['SPY_log_ret'].dropna()

adf_res = run_adf(y_ret, autolag='AIC')
pp_res  = run_pp(y_ret, trend='c')

# ---------- Print Results ----------
print("=== Augmented Dickey–Fuller (ADF) on SPY_log_ret ===")
print(f"Test statistic: {adf_res['stat']:.4f} | p-value: {adf_res['pvalue']:.4g} "
      f"| lags: {adf_res['lags_used']} | nobs: {adf_res['nobs']}")
print(f"Critical values: 1%={adf_res['crit']['1%']:.3f}, 5%={adf_res['crit']['5%']:.3f}, 10%={adf_res['crit']['10%']:.3f}")
print("Verdict (ADF):", stationarity_verdict(adf_res['pvalue']))

print("\n=== Phillips–Perron (PP) on SPY_log_ret ===")
print(f"Test statistic: {pp_res['stat']:.4f} | p-value: {pp_res['pvalue']:.4g} | lags: {pp_res['lags_used']}")
print(f"Critical values: { {k: round(v,3) for k,v in pp_res['crit'].items()} }")
print("Verdict (PP): ", stationarity_verdict(pp_res['pvalue']))



=== Augmented Dickey–Fuller (ADF) on SPY_log_ret ===
Test statistic: -17.7352 | p-value: 3.421e-30 | lags: 15 | nobs: 4879
Critical values: 1%=-3.432, 5%=-2.862, 10%=-2.567
Verdict (ADF): Reject unit root (stationary)

=== Phillips–Perron (PP) on SPY_log_ret ===
Test statistic: -78.0260 | p-value: 0 | lags: 32
Critical values: {'1%': -3.432, '5%': -2.862, '10%': -2.567}
Verdict (PP):  Reject unit root (stationary)


# Long- Memory Test (R/S & GPH)

In [29]:
# ===============================================================
# Long-Memory Tests on SPY_log_ret
# - Rescaled Range (R/S) -> Hurst exponent H
# - Geweke–Porter–Hudak (GPH) -> fractional differencing d
# ===============================================================
import numpy as np
import pandas as pd

# ---------- Helper Functions ----------
def hurst_rs(x: pd.Series, min_window=16, max_window=None):
    """
    Classic R/S estimator of Hurst exponent H.
    H ~ 0.5 short memory; H>0.5 persistence; H<0.5 anti-persistence.
    """
    x = pd.Series(x).dropna().values.astype(float)
    n = x.size
    if n < 64:
        raise ValueError("Series too short for R/S estimation.")
    if max_window is None:
        max_window = n // 4
    wins = np.unique(np.floor(np.logspace(np.log10(min_window), np.log10(max_window), num=12)).astype(int))
    RS = []
    for w in wins:
        if w < 8: 
            continue
        blocks = n // w
        if blocks < 2:
            continue
        vals = []
        for b in range(blocks):
            seg = x[b*w:(b+1)*w]
            seg = seg - seg.mean()
            y = np.cumsum(seg)
            R = y.max() - y.min()
            S = seg.std(ddof=1)
            if S > 0:
                vals.append(R / S)
        if len(vals):
            RS.append((w, np.mean(vals)))
    RS = np.array(RS)
    slope = np.polyfit(np.log(RS[:,0]), np.log(RS[:,1]), 1)[0]
    return {'H': slope, 'windows': RS[:,0], 'RS': RS[:,1]}

def gph_fractional_d(x: pd.Series, m=None):
    """
    GPH (log-periodogram) estimator of fractional differencing parameter d.
    Uses the regression: log I(λ_j) ~ a - d * log(4 sin^2(λ_j/2)) + u_j
    => slope ≈ -d, so d_hat = -slope
    """
    x = pd.Series(x).dropna().values.astype(float)
    x = x - x.mean()
    T = len(x)
    if m is None:
        m = int(T**0.5)  # common choice; try T^0.6 for robustness
    j = np.arange(1, m+1)
    lam = 2*np.pi*j/T
    X = np.fft.fft(x)
    I = (1/(2*np.pi*T)) * (np.abs(X[j])**2)
    z = np.log(4*(np.sin(lam/2)**2))
    y = np.log(I)
    b = np.polyfit(z, y, 1)[0]  # slope
    d_hat = -b
    zc = z - z.mean()
    se = np.pi / np.sqrt(6*np.sum(zc**2))  # asymptotic SE (approx)
    ci = (d_hat - 1.96*se, d_hat + 1.96*se)
    return {'d': d_hat, 'se': se, 'ci95': ci, 'm': m, 'T': T}

def d_comment(d):
    if d < 0:
        return "Anti-persistent / over-differenced"
    if 0 <= d < 0.5:
        return "Long memory but covariance-stationary (ARFIMA range)"
    if 0.5 <= d < 1:
        return "Non-stationary long memory (mean-reverting, infinite variance)"
    return "Unit root or stronger non-stationarity"

# ---------- Run Tests ----------
y_ret = df['SPY_log_ret'].dropna()

rs_res  = hurst_rs(y_ret, min_window=16)
gph_res = gph_fractional_d(y_ret, m=None)  # defaults to m = T^0.5

# ---------- Print Results ----------
print("=== Rescaled Range (R/S) — Hurst Exponent ===")
print(f"H (R/S): {rs_res['H']:.3f}  ->  "
      f"{'Long memory (persistent)' if rs_res['H']>0.5 else ('Anti-persistent' if rs_res['H']<0.5 else 'Short memory')}")

print("\n=== GPH Log-Periodogram — Fractional d ===")
print(f"d_hat: {gph_res['d']:.3f}  |  SE≈{gph_res['se']:.3f}  |  95% CI: "
      f"{tuple(round(x,3) for x in gph_res['ci95'])}  |  m={gph_res['m']} of T={gph_res['T']}")
print("Interpretation (d):", d_comment(gph_res['d']))

=== Rescaled Range (R/S) — Hurst Exponent ===
H (R/S): 0.542  ->  Long memory (persistent)

=== GPH Log-Periodogram — Fractional d ===
d_hat: -0.023  |  SE≈0.086  |  95% CI: (-0.19, 0.145)  |  m=69 of T=4895
Interpretation (d): Anti-persistent / over-differenced


# Long-Memory Diagnostics on Volatility Series

In [31]:
# ===============================================================
# Long-Memory Diagnostics on Volatility Proxy
# - Absolute and Squared Returns
# ===============================================================
# Use your return series
y_ret = df['SPY_log_ret'].dropna()

# Create volatility proxies
abs_ret = np.abs(y_ret)
sq_ret  = y_ret ** 2

print(f"Absolute returns mean: {abs_ret.mean():.4f}, variance: {abs_ret.var():.4e}")
print(f"Squared returns mean: {sq_ret.mean():.4e}, variance: {sq_ret.var():.4e}")

# ---------------------------------------------------------------
# Reuse Hurst and GPH functions from previous cells
# ---------------------------------------------------------------
def hurst_rs(x: pd.Series, min_window=16, max_window=None):
    x = pd.Series(x).dropna().values.astype(float)
    n = x.size
    if n < 64:
        raise ValueError("Series too short for R/S estimation.")
    if max_window is None:
        max_window = n // 4
    wins = np.unique(np.floor(np.logspace(np.log10(min_window), np.log10(max_window), num=12)).astype(int))
    RS = []
    for w in wins:
        if w < 8: 
            continue
        blocks = n // w
        if blocks < 2:
            continue
        vals = []
        for b in range(blocks):
            seg = x[b*w:(b+1)*w]
            seg = seg - seg.mean()
            y = np.cumsum(seg)
            R = y.max() - y.min()
            S = seg.std(ddof=1)
            if S > 0:
                vals.append(R / S)
        if len(vals):
            RS.append((w, np.mean(vals)))
    RS = np.array(RS)
    slope = np.polyfit(np.log(RS[:,0]), np.log(RS[:,1]), 1)[0]
    return {'H': slope, 'windows': RS[:,0], 'RS': RS[:,1]}

def gph_fractional_d(x: pd.Series, m=None):
    x = pd.Series(x).dropna().values.astype(float)
    x = x - x.mean()
    T = len(x)
    if m is None:
        m = int(T**0.5)
    j = np.arange(1, m+1)
    lam = 2*np.pi*j/T
    X = np.fft.fft(x)
    I = (1/(2*np.pi*T)) * (np.abs(X[j])**2)
    z = np.log(4*(np.sin(lam/2)**2))
    y = np.log(I)
    b = np.polyfit(z, y, 1)[0]
    d_hat = -b
    zc = z - z.mean()
    se = np.pi / np.sqrt(6*np.sum(zc**2))
    ci = (d_hat - 1.96*se, d_hat + 1.96*se)
    return {'d': d_hat, 'se': se, 'ci95': ci, 'm': m, 'T': T}

def d_comment(d):
    if d < 0:
        return "Anti-persistent / over-differenced"
    if 0 <= d < 0.5:
        return "Long memory but covariance-stationary (FIEGARCH range)"
    if 0.5 <= d < 1:
        return "Non-stationary long memory (high persistence)"
    return "Unit root or stronger non-stationarity"

# ---------------------------------------------------------------
# Run diagnostics on |returns| and squared returns
# ---------------------------------------------------------------
for label, series in {"|r_t|": abs_ret, "r_t^2": sq_ret}.items():
    print(f"\n=== Long-Memory Diagnostics for {label} ===")
    
    rs_res = hurst_rs(series)
    gph_res = gph_fractional_d(series)
    
    print(f"H (R/S): {rs_res['H']:.3f}  ->  "
          f"{'Long memory (persistent)' if rs_res['H']>0.5 else ('Anti-persistent' if rs_res['H']<0.5 else 'Short memory')}")
    
    print(f"d_hat: {gph_res['d']:.3f}  |  SE≈{gph_res['se']:.3f}  |  95% CI: "
          f"{tuple(round(x,3) for x in gph_res['ci95'])}  |  m={gph_res['m']} of T={gph_res['T']}")
    print("Interpretation:", d_comment(gph_res['d']))


Absolute returns mean: 0.7876, variance: 9.0095e-01
Squared returns mean: 1.5210e+00, variance: 3.7276e+01

=== Long-Memory Diagnostics for |r_t| ===
H (R/S): 0.885  ->  Long memory (persistent)
d_hat: 0.435  |  SE≈0.086  |  95% CI: (0.267, 0.603)  |  m=69 of T=4895
Interpretation: Long memory but covariance-stationary (FIEGARCH range)

=== Long-Memory Diagnostics for r_t^2 ===
H (R/S): 0.831  ->  Long memory (persistent)
d_hat: 0.296  |  SE≈0.086  |  95% CI: (0.128, 0.464)  |  m=69 of T=4895
Interpretation: Long memory but covariance-stationary (FIEGARCH range)


# Model Specification: Determine optimal orders for the ARIMAX (p,d,q) model and incorporate DI and WTI Oil returns as exogenous variables

In [33]:
# ===============================================================
# ARIMAX(p,d,q) with exogenous DXY_ret & WTI_ret
# - auto_arima on train
# - OOS preds for val/test
# - residuals for volatility modeling
# ===============================================================
import pmdarima as pm

import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")
warnings.filterwarnings("ignore", category=FutureWarning, module="pmdarima")

TARGET = 'SPY_log_ret'
EXOG   = ['DXY_ret', 'WTI_ret']

def fit_arimax_auto(train_df, val_df=None, test_df=None, target=TARGET, exog_cols=EXOG):
    y_tr   = train_df[target].dropna()
    X_tr   = train_df[exog_cols].loc[y_tr.index]

    model = pm.auto_arima(
        y_tr, exogenous=X_tr, seasonal=False,
        start_p=0, start_q=0, max_p=6, max_q=6, max_d=2,
        information_criterion='aic', stepwise=True, trace=False,
        error_action='ignore', suppress_warnings=True
    )
    print("Best ARIMAX order:", model.order, "| AIC:", round(model.aic(), 2))

    out = {'model': model}

    # In-sample fitted + residuals (train)
    y_hat_tr = pd.Series(model.predict_in_sample(exogenous=X_tr), index=y_tr.index, name='yhat_tr')
    resid_tr = (y_tr - y_hat_tr).rename('resid_tr')
    out.update({'yhat_tr': y_hat_tr, 'resid_tr': resid_tr})

    # Validation OOS preds
    if val_df is not None:
        X_va = val_df[exog_cols]
        y_hat_va = pd.Series(model.predict(n_periods=len(val_df), exogenous=X_va), index=val_df.index, name='yhat_va')
        out['yhat_va'] = y_hat_va

    # Test OOS preds
    if test_df is not None:
        X_te = test_df[exog_cols]
        y_hat_te = pd.Series(model.predict(n_periods=len(test_df), exogenous=X_te), index=test_df.index, name='yhat_te')
        out['yhat_te'] = y_hat_te

    return out

arimax_out = fit_arimax_auto(train_df, val_df, test_df)
# Collect residuals for volatility model: use train (and optionally append val)
resid_for_vol = arimax_out['resid_tr']  # + you can append (val - yhat_va) if you prefer
print("Residuals prepared for volatility model:", resid_for_vol.shape)


Best ARIMAX order: (2, 0, 3) | AIC: 12806.21
Residuals prepared for volatility model: (3916,)


C:\Users\josep\anaconda3\envs\LSTM_Research_Project\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\josep\anaconda3\envs\LSTM_Research_Project\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\josep\anaconda3\envs\LSTM_Research_Project\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\josep\anaconda3\envs\LSTM_Research_Project\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version,

# Check ARIMAX residuals for ARCH effects

In [35]:
# Check ARIMAX residuals for ARCH effects
from statsmodels.stats.diagnostic import het_arch
from statsmodels.stats.diagnostic import acorr_ljungbox

resid = arimax_out['resid_tr'].dropna()

# Ljung–Box on residuals (autocorrelation in mean)
lb = acorr_ljungbox(resid, lags=[10,20], return_df=True)

# Engle ARCH LM test (heteroskedasticity)
arch_stat, arch_pvalue, _, _ = het_arch(resid, maxlag=12)

print("Ljung–Box p-values (resid):")
print(lb['lb_pvalue'])
print(f"\nARCH LM p-value (lag 12): {arch_pvalue:.4g}  -> "
      f"{'ARCH present' if arch_pvalue < 0.05 else 'No strong ARCH'}")


Ljung–Box p-values (resid):
10    0.033076
20    0.000027
Name: lb_pvalue, dtype: float64

ARCH LM p-value (lag 12): 1.448e-258  -> ARCH present


C:\Users\josep\AppData\Local\Temp\ipykernel_76596\2476468035.py:11: FutureWarning: the 'maxlag' keyword is deprecated, use 'nlags' instead.
  arch_stat, arch_pvalue, _, _ = het_arch(resid, maxlag=12)


# Utilise Python proxy for model estimation: FIGARCH (long memory) or EGARCH (asymmetry)

In [41]:
from arch.univariate import arch_model

resid = arimax_out['resid_tr'].dropna()

# Choose the proxy model
VOL_KIND = 'FIGARCH'   # 'FIGARCH' (captures long memory d) or 'EGARCH' (captures asymmetry)
DIST     = 't'

if VOL_KIND.upper() == 'EGARCH':
    am = arch_model(resid, mean='Zero', vol='EGARCH', p=1, o=1, q=1, dist=DIST)
elif VOL_KIND.upper() == 'FIGARCH':
    am = arch_model(resid, mean='Zero', vol='FIGARCH', p=1, q=1, dist=DIST)
else:
    raise ValueError("VOL_KIND must be 'EGARCH' or 'FIGARCH'")

vol_res = am.fit(disp='off')
print(vol_res.summary())

# Forecast variance for the validation window
h_va = len(val_df)
var_fc_va = vol_res.forecast(horizon=h_va, reindex=False).variance.values.ravel()
var_fc_va = pd.Series(var_fc_va, index=val_df.index, name=f'var_fc_{VOL_KIND.lower()}')
print("Variance forecasts shape:", var_fc_va.shape)

                         Zero Mean - FIGARCH Model Results                          
Dep. Variable:                     resid_tr   R-squared:                       0.000
Mean Model:                       Zero Mean   Adj. R-squared:                  0.000
Vol Model:                          FIGARCH   Log-Likelihood:               -5074.10
Distribution:      Standardized Student's t   AIC:                           10158.2
Method:                  Maximum Likelihood   BIC:                           10189.6
                                              No. Observations:                 3916
Date:                      Fri, Nov 07 2025   Df Residuals:                     3916
Time:                              14:58:08   Df Model:                            0
                              Volatility Model                              
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omeg

# Compare EGARCH and FIGARCH

In [43]:
# ===========================
# Compare EGARCH vs FIGARCH
# ===========================
import warnings
warnings.filterwarnings("ignore")

from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch

resid = arimax_out['resid_tr'].dropna()

def fit_egarch(resid, dist='t'):
    m = arch_model(resid, mean='Zero', vol='EGARCH', p=1, o=1, q=1, dist=dist)
    r = m.fit(disp='off')
    # pull leverage coefficient (gamma[1]) and p-value
    params = r.params
    pvals  = r.pvalues
    # arch names are typically: omega, alpha[1], gamma[1], beta[1], (nu)...
    gamma = params.get('gamma[1]', np.nan)
    gamma_p = pvals.get('gamma[1]', np.nan)
    out = dict(model='EGARCH(1,1)', loglik=r.loglikelihood, aic=r.aic, bic=r.bic,
               gamma=gamma, gamma_p=gamma_p, res=r)
    return out

def fit_figarch(resid, dist='t', p=1, q=1):
    m = arch_model(resid, mean='Zero', vol='FIGARCH', p=p, q=q, dist=dist)
    r = m.fit(disp='off')
    d = r.params.get('d', np.nan)
    d_p = r.pvalues.get('d', np.nan)
    out = dict(model=f'FIGARCH({p},d,{q})', loglik=r.loglikelihood, aic=r.aic, bic=r.bic,
               d=d, d_p=d_p, res=r)
    return out

eg = fit_egarch(resid, dist='t')
fi = fit_figarch(resid, dist='t', p=1, q=1)   # you can also try q=0 if phi was insignificant

# Summary table
cmp = pd.DataFrame([
    {'Model': eg['model'], 'LogLik': eg['loglik'], 'AIC': eg['aic'], 'BIC': eg['bic'],
     'Leverage gamma': eg['gamma'], 'gamma p': eg['gamma_p'], 'd (FIGARCH)': np.nan, 'd p': np.nan},
    {'Model': fi['model'], 'LogLik': fi['loglik'], 'AIC': fi['aic'], 'BIC': fi['bic'],
     'Leverage gamma': np.nan, 'gamma p': np.nan, 'd (FIGARCH)': fi['d'], 'd p': fi['d_p']}
]).set_index('Model')
print(cmp.round(4))

# Residual diagnostics for each model
def diag(name, res_fit):
    std = res_fit.std_resid.dropna()
    lb = acorr_ljungbox(std, lags=[10,20], return_df=True)['lb_pvalue']
    arch_p = het_arch(std, nlags=12)[1]  # use nlags (not maxlag)
    print(f"\n{name} standardized residual checks:")
    print(f"  Ljung–Box p (10,20): {lb.iloc[0]:.4f}, {lb.iloc[1]:.4f}")
    print(f"  ARCH LM p (lag 12): {arch_p:.4g}")

diag('EGARCH', eg['res'])
diag('FIGARCH', fi['res'])

# Simple decision hints
better = 'EGARCH' if eg['aic'] < fi['aic'] else 'FIGARCH'
print(f"\nLower AIC -> {better}")
if not np.isnan(eg['gamma_p']):
    print(f"EGARCH leverage gamma={eg['gamma']:.4f}, p={eg['gamma_p']:.4g} "
          f"-> {'material (leverage present)' if eg['gamma_p']<0.05 else 'not material'} "
          f"{'(negative -> leverage)' if eg['gamma']<0 else '(positive)'}")

                   LogLik         AIC         BIC  Leverage gamma  gamma p  \
Model                                                                        
EGARCH(1,1)    -5003.1286  10016.2571  10047.6213         -0.1673      0.0   
FIGARCH(1,d,1) -5074.0969  10158.1938  10189.5579             NaN      NaN   

                d (FIGARCH)  d p  
Model                             
EGARCH(1,1)             NaN  NaN  
FIGARCH(1,d,1)       0.6311  0.0  

EGARCH standardized residual checks:
  Ljung–Box p (10,20): 0.0004, 0.0005
  ARCH LM p (lag 12): 0.8115

FIGARCH standardized residual checks:
  Ljung–Box p (10,20): 0.0000, 0.0001
  ARCH LM p (lag 12): 0.06821

Lower AIC -> EGARCH
EGARCH leverage gamma=-0.1673, p=1.444e-30 -> material (leverage present) (negative -> leverage)
